<a href="https://colab.research.google.com/github/Firdavs222/Portfolio-projects/blob/main/Faster_whisper_transcripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google drive ma'lumotlaridan foydalanish uchun Google akkauntni sinxronlandi.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Endi kerakli bo'ladigan kutubxonalarni "!pip" buyrug'i bilan yuklab olinadi. Bu kutubxonalar faster-whisper(ya'ni openai-whisper ASR ning 4 baravar tezlashtirilgan modelidir) va jiwer(ASR modeldan qaytgan transkripsiya va haqiqiy transkripsiyalarni solishtirish uchun) kutubxonalaridir.

In [2]:
!pip install faster-whisper
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.8 MB/s eta 0:00:00


Keyingi bosqichda drivedan yuklangan faylni "./unzipped_data" papkasiga extract qilindi.

In [3]:
import tarfile

file_name = "/content/drive/MyDrive/cv-corpus-12-delta-2022-12-07.tar.gz"

output_dir = "./unzipped_data"

with tarfile.open(file_name, "r") as tar:
    tar.extractall(path=output_dir)


Endi faster-whisper kutubxonasining WhisperModel classining small ya'ni kichik modelidan foydalanib fayldagi ovozlarning transkripsiyalarini aniqladim va ularni audio_with_transcript lug'atini yaratib kalitga(key) ovozlar nomini va qiymatga(value) transkripsiyalarni saqladim.

In [4]:
from faster_whisper import WhisperModel
import os

model_size = "small"

model = WhisperModel(model_size)
clip_dir = "/content/unzipped_data/Downloads/cv-corpus-12.0-delta-2022-12-07-uz/cv-corpus-12.0-delta-2022-12-07/Clips"
clip_names = os.listdir(clip_dir)

audio_with_transcript = {}

for clip_name in clip_names:
    clip_path = os.path.join(clip_dir,clip_name)
    segments, info = model.transcribe(
    clip_path,
    language="uz",
    )
    segments=list(segments)
    audio_with_transcript[clip_name]=segments


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

Bu qismda esa haqiqiy transkripsiyalarni nomlari bilan birga real_transcriptions lug'atiga saqladim.

In [5]:
import os
import pandas as pd

real_transcriptions = {}
file_names = ["/content/unzipped_data/Downloads/cv-corpus-12.0-delta-2022-12-07-uz/cv-corpus-12.0-delta-2022-12-07/invalidated.tsv",
              "/content/unzipped_data/Downloads/cv-corpus-12.0-delta-2022-12-07-uz/cv-corpus-12.0-delta-2022-12-07/other.tsv",
              "/content/unzipped_data/Downloads/cv-corpus-12.0-delta-2022-12-07-uz/cv-corpus-12.0-delta-2022-12-07/validated.tsv"]
for file_name in file_names:
    df = pd.read_csv(file_name,sep="\t")
    for _, row in df.iterrows():
        clip_name = row["path"]
        sentence = row["sentence"]
        real_transcriptions[clip_name] = sentence



In [16]:
real_transcriptions[clip_name]

'Daho boshi bilan mehmonlarni ko‘rsatdi.'

Bu bosqichda modelning performansi ya'ni samaradorligini aniqlash uchun 'jiwer' kutubxonasining 'cer' classidan foydalandim ('cer' bu "Character Error Rate" hisoblanadi). 'cers' degan bo'sh ro'yxat yaratib uning ichiga 'cer'larni qo'shib chiqib o'rtacha umumiy CER ni hisobladim.

In [26]:
from jiwer import cer

cers = []

for clip in audio_with_transcript.keys():
    if clip in real_transcriptions.keys():
        asr_transcript = [segment.text for segment in audio_with_transcript[clip_name]]
        CER = cer(real_transcriptions[clip_name],asr_transcript)
        cers.append(CER)
avg_cer = sum([n for n in cers])/len(cers)

In [28]:
print(avg_cer)
print(len(cers))

0.2564102564102682
2683
